# Importazione dei dati

Viene preso il file "DB-Output_original.csv", e tramite la libreria "pandas" viene importato all'interno di un dataframe (sul quale verranno effettuati tutti i lavori). 

La tabella viene ridotta da 35mila righe a 9mila, in modo da velocizzare l'esecuzione di alcuni passaggi, senza però perdere informazioni importanti nell'analisi.

Vengono inoltre eliminate tutte le colonne non importanti, mantenendone soltanto quattro utili al raggiungimento dello scopo finale.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
path = "DB-Output_original.csv"

data = pd.read_csv(path)

data = data[:9000] # posso ridurre da 35mila a 9mila senza perdere i luoghi, velocizzando l'esecuzione
data = data.loc[:, :'OF_Diff']

data

,Instance's Name,Initial Solution,OFIS,Moves,OFFS,OF_Diff
0,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...",30978.64707,"['null', 'null', 'DemandBasedDestroyCustomer',...",30978.64707,0.000000
1,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...",30978.64707,"['null', 'null', 'ShawDestroyCustomer', 'Greed...",30978.64707,0.000000
2,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...",30978.64707,"['null', 'null', 'WorstTimeDestroyCustomer', '...",30978.64707,0.000000
3,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...",30978.64707,"['null', 'null', 'GreedyRouteRemoval', 'Greedy...",30978.64707,0.000000
4,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...",30978.64707,"['RandomDestroyStation', 'ProbabilisticBestRep...",30978.64707,0.000000
...,...,...,...,...,...,...
8995,c104_21_100.txt,"[['D0', 'C21', 'C22', 'C66', 'D0'], ['D0', 'C6...",47954.67971,"['null', 'null', 'DemandBasedDestroyCustomer',...",47210.04063,744.639080
8996,c104_21_100.txt,"[['D0', 'C21', 'C22', 'C66', 'D0'], ['D0', 'C2...",47210.04063,"['null', 'null', 'RandomRouteDestroyCustomer',...",45999.65256,1210.388073
8997,c104_21_100.txt,"[['D0', 'C21', 'C22', 'C66', 'D0'], ['D0', 'C2...",45999.65256,"['LongestWaitingTimeDestroyStation', 'Probabil...",45999.65256,0.000000
8998,c104_21_100.txt,"[['D0', 'C21', 'C22', 'C66', 'D0'], ['D0', 'C2...",45999.65256,"['null', 'null', 'ZoneDestroyCustomer', 'Greed...",45996.14260,3.509958


In [2]:
print(data["Moves"])

0       ['null', 'null', 'DemandBasedDestroyCustomer',...
1       ['null', 'null', 'ShawDestroyCustomer', 'Greed...
2       ['null', 'null', 'WorstTimeDestroyCustomer', '...
3       ['null', 'null', 'GreedyRouteRemoval', 'Greedy...
4       ['RandomDestroyStation', 'ProbabilisticBestRep...
                              ...                        
8995    ['null', 'null', 'DemandBasedDestroyCustomer',...
8996    ['null', 'null', 'RandomRouteDestroyCustomer',...
8997    ['LongestWaitingTimeDestroyStation', 'Probabil...
8998    ['null', 'null', 'ZoneDestroyCustomer', 'Greed...
8999    ['null', 'null', 'GreedyRouteRemoval', 'Greedy...
Name: Moves, Length: 9000, dtype: object


L'analisi si basa sulla creazione di una matrice di adicienza per ogni percorso all'interno della colonna "Initial Solution". Ogni percorso è inteso come una stringa contenente diversi luoghi, che parte da "D0" (il deposito), presenta altri luoghi (es. C21, C65, ecc...), e termina di nuovo in "D0".

La colonna più interessante per lo scopo è quella rappresentata da "Initial Solution", che quindi viene isolata e inserita all'interno del vettore "init_sol".

Ogni colonna del dataframe iniziale è una Series (della libreria "pandas"), al cui interno è contenuta una stringa testuale.

In [3]:
init_sol = data['Initial Solution']
init_sol.head()

0    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
1    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
2    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
3    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
4    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
Name: Initial Solution, dtype: object

Per creare n matrici di adiacenza, si parte dall'isolamento dei percorsi contenuti nella variabile "init_sol". Vengono quindi splittate le righe contenenti i percorsi, secondo i caratteri "']", e il risultato di questi split viene inserito in una lista chiamata "sol". 

La lista "sol" quindi conterrà i percorsi ricavati, dove ogni indice della lista è occupato da un diverso percorso. La lista viene poi ripulita da eventuli altri caratteri di punteggiatura tramite la funzione "replace()", in modo da avere i percorsi in chiaro.

In [4]:
sol = []
for row in init_sol:
    for elem in row.split("'], "):
        sol.append(elem)
        
for i in range(len(sol)):
    sol[i] = sol[i].replace("[['","")
    sol[i] = sol[i].replace("']]","")
    sol[i] = sol[i].replace("'","")
    sol[i] = sol[i].replace(",","")
    sol[i] = sol[i].replace("[","")

In [5]:
sol[:10]

['D0 C21 C26 D0',
 'D0 C65 C61 D0',
 'D0 C24 C50 D0',
 'D0 C25 D0',
 'D0 C6 C91 D0',
 'D0 C9 D0',
 'D0 C2 D0',
 'D0 C90 C85 D0',
 'D0 C32 S11 D0',
 'D0 C82 C81 S20 D0']

# Divisione dei luoghi

La matrice di adiacenza di ogni percorso deve avere come indici i singoli luoghi, divisi, messi in ordine alfabetico e senza ripetizioni.

Si "isola" quindi nuovamente la colonna "Initial Solution", inserendola nella variabile "init_sol". Questo passaggio viene fatto per ripulire la colonna "Initial Solution", splittarla rispetto ad un carattere spazio, ed inserire il risultato dello split in una lista che conterrà tutti i singoli luoghi presenti nel dataframe iniziale. Successivamente, questa lista verrà lavorata per eliminare le ripetizioni ed ottenere i singli luoghi senza ripetizioni.

In [6]:
data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
data["Initial Solution"] = data["Initial Solution"].str.replace("]","")
data["Initial Solution"] = data["Initial Solution"].str.replace(",","")
data["Initial Solution"] = data["Initial Solution"].str.replace("'","")

init_sol = data["Initial Solution"]

print(init_sol)

0       D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
1       D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
2       D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
3       D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
4       D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
                              ...                        
8995    D0 C21 C22 C66 D0 D0 C67 C65 D0 D0 C23 C26 C28...
8996    D0 C21 C22 C66 D0 D0 C23 C26 C28 C30 D0 D0 C63...
8997    D0 C21 C22 C66 D0 D0 C23 C26 C28 C30 D0 D0 C63...
8998    D0 C21 C22 C66 D0 D0 C23 C26 C28 C30 D0 D0 C63...
8999    D0 C21 C22 C66 D0 D0 C23 C26 C28 C30 D0 D0 C63...
Name: Initial Solution, Length: 9000, dtype: object


/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_73423/787485621.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_73423/787485621.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Initial Solution"] = data["Initial Solution"].str.replace("]","")


La singola riga di "init_sol" equivale ad una stringa. Ad esempio "init_sol[0]" equivale ad una stringa contenente "D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 ..."

Attraverso il metodo "split()", inserito in un ciclo che itera ogni riga di "init_sol", si splitta la stringa in considerazione rispetto al carattere spazio e la si inserisce tramite ".append()" nella lista "places", inizialmente vuota. Ogni volta che il metodo "split()" trova uno spazio, viene rilevato un elemento (luogo).

In [7]:
places = []
for row in init_sol:
    for elem in row.split():
        places.append(elem)

Si converte la lista di tutti i luoghi (con ripetizione) in un array Numpy chiamato "places".

In [8]:
places = np.array(places)
print(places[:25])

['D0' 'C21' 'C26' 'D0' 'D0' 'C65' 'C61' 'D0' 'D0' 'C24' 'C50' 'D0' 'D0'
 'C25' 'D0' 'D0' 'C6' 'C91' 'D0' 'D0' 'C9' 'D0' 'D0' 'C2' 'D0']


Tramite il metodo "np.unique(places)", si prende il vettore "places" e si crea un ulteriore vettore contenente solo 120 luoghi, senza ripetizioni e che servirà come indice del dataframe, chiamato "distinct_places"

In [9]:
distinct_places = np.unique(places)

print("\nLa lista di tutti i diversi luoghi (ordinati) è:\n", distinct_places)
print("\nLa lista contiene", len(distinct_places), "diversi luoghi")


La lista di tutti i diversi luoghi (ordinati) è:
 ['C1' 'C10' 'C100' 'C11' 'C12' 'C13' 'C14' 'C15' 'C16' 'C17' 'C18' 'C19'
 'C2' 'C20' 'C21' 'C22' 'C23' 'C24' 'C25' 'C26' 'C27' 'C28' 'C29' 'C3'
 'C30' 'C31' 'C32' 'C33' 'C34' 'C35' 'C36' 'C37' 'C38' 'C39' 'C4' 'C40'
 'C41' 'C42' 'C43' 'C44' 'C45' 'C46' 'C47' 'C48' 'C49' 'C5' 'C50' 'C51'
 'C52' 'C53' 'C54' 'C55' 'C56' 'C57' 'C58' 'C59' 'C6' 'C60' 'C61' 'C62'
 'C63' 'C64' 'C65' 'C66' 'C67' 'C68' 'C69' 'C7' 'C70' 'C71' 'C72' 'C73'
 'C74' 'C75' 'C76' 'C77' 'C78' 'C79' 'C8' 'C80' 'C81' 'C82' 'C83' 'C84'
 'C85' 'C86' 'C87' 'C88' 'C89' 'C9' 'C90' 'C91' 'C92' 'C93' 'C94' 'C95'
 'C96' 'C97' 'C98' 'C99' 'D0' 'S0' 'S1' 'S10' 'S11' 'S12' 'S13' 'S14'
 'S15' 'S16' 'S17' 'S18' 'S19' 'S2' 'S20' 'S3' 'S4' 'S5' 'S7' 'S9']

La lista contiene 120 diversi luoghi


# Creazione della matrice 

Si procede quindi alla creazione di un dizionario "dict" contenente un dataframe (pieno di zeri) per ogni percorso, che prende da:

- "distinct_places" gli indici, secondo i quali andare ad inserire le corrispondenze
- "zeros" le dimensioni di 120x120, date dalla lunghezza del vettore "distinct_places", ovvero il numero dei diversi luoghi presenti all'interno del vettore.

In [11]:
# creazione dizionario  
dict = {}

# creazione dataframe per ogni indice del dizionario, dove l'indice è la lunghezza di "sol"
for index, elem in enumerate(sol):
        zeros = np.zeros([len(distinct_places), len(distinct_places)])
        dict[index] = pd.DataFrame(zeros, index=distinct_places, columns=distinct_places)


Si prende quindi la lista di partenza "sol" (la lista contenente tutti i percorsi del dataframe di partenza), e tramite un ciclo for che itera tutta la lista, si prende ogni percorso e si inseriscono all'interno della matrice di adiacenza corrispondente tutti gli 1 necessari.

Più nel dettaglio:
- si prende la stringa i-esima e la si mette in una variabile temporanea chiamata "temp", che diventerà percorso 1, percorso 2, ecc...
- si splitta la variabile "temp", e quindi il percorso, secondo il carattere spazio, inserendo ogni singolo luogo all'interno del percorso in una variabile chiamata "split" (che sarà una lista), in modo da avere ad esempio split[0] = D0, split[1] = C21, ecc...
- la lista "split" viene convertita in un array Numpy chiamato allo stesso modo
- tramite un ciclo for innestato, si prende ogni indice del dizionario che contiene un dataframe, e si va ad inserire il numero "1" a seconda dei luoghi presenti all'interno del percorso considerato. Es. percorso 0 = "D0 C21 C26 D0", prendo il dataframe contenuto all'indice 0 del dizionario e inserisco le corrispondenze. Quando si finisce con un percorso si passa al successivo, prendendo quindi anche il successivo dataframe del dizionario.

In [12]:
for index, stringa in enumerate(sol):
    
    temp = stringa
    
    split = temp.split(" ")                         
    split = np.array(split)
    
    for i in range(len(split)-1):
            dict[index].loc[split[i], split[i+1]] = 1

Infine, si converte ogni dataframe contenuto nel dizionario in una matrice (di adiacenza) Numpy.

In [13]:
dict[0]

,C1,C10,C100,C11,C12,C13,C14,C15,C16,C17,C18,C19,C2,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C3,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C4,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C5,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C6,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C7,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C8,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C9,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,D0,S0,S1,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S2,S20,S3,S4,S5,S7,S9
C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [14]:
for i in range(len(dict)):
    dict[i] = dict[i].to_numpy()